### import 불러오기

In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
import csv
import time
from tqdm import tqdm


### 1곡 정보 불러오기 테스트

In [ ]:
# 웹 드라이버 설정
driver = webdriver.Chrome()

# 멜론 웹사이트 열기
driver.get("http://www.melon.com/genre/song_list.htm?gnrCode=GN0300")

# 페이지 로딩을 위해 잠시 대기
time.sleep(3)  # 페이지가 완전히 로드될 때까지 기다림

# 가사 페이지로 이동
lyric_page = driver.find_element(By.XPATH, "//a[@class='btn button_icons type03 song_info']")
lyric_page.click()

# 페이지 로딩을 위해 잠시 대기
time.sleep(3)  # 페이지가 완전히 로드될 때까지 기다림

# 제목, 발매일, 좋아요, 가수, 가사 정보 추출
title = driver.find_element(By.XPATH, "//div[@class='song_name']")
print("제목 :", title.text)

date = driver.find_element(By.XPATH, "//dt[text()='발매일']/following-sibling::dd")
print("발매일 :", date.text)

likes = driver.find_element(By.XPATH, "//span[@id='d_like_count']")
print("좋아요 :", likes.text)

singer = driver.find_element(By.XPATH, "//a[@class='artist_name']")
print("가수 :", singer.text)

lyrics = driver.find_element(By.XPATH, "//div[@class='lyric']")
print("가사 :")
print(lyrics.text)

### 100페이지 수집

In [ ]:
# 웹 드라이버 설정
driver = webdriver.Chrome()

# CSV 파일 열기
with open('song_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 헤더 작성
    writer.writerow(["title", "artist", "like_cnt", "lyrics", "release_date"])

    # 총 100페이지를 반복합니다.
    for page in tqdm(range(100), desc="페이지 진행 상황"):
        start_index = page * 50 + 1
        url = f"http://www.melon.com/genre/song_list.htm?gnrCode=GN0400#params%5BgnrCode%5D=GN0400&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&po=pageObj&startIndex={start_index}"
        driver.get(url)
        time.sleep(5)

        try:
            song_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[@class='btn button_icons type03 song_info']"))
            )
        except TimeoutException:
            print("Song elements loading timed out.")
            continue

        for song_index in tqdm(range(len(song_elements)), desc="노래 진행 상황", leave=False):
            try:
                # 노래 정보 클릭
                song_elements[song_index].click()

                # 노래 정보 추출
                title = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='song_name']"))
                ).text
                singer = driver.find_element(By.XPATH, "//a[@class='artist_name']").text
                likes = driver.find_element(By.XPATH, "//span[@id='d_like_count']").text

                # "펼치기" 버튼 클릭 시도
                try:
                    expand_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@class='button_more arrow_d' and @data-expose-target='#d_video_summary']"))
                    )
                    expand_button.click()
                    # 가사 영역이 확장되었는지 확인
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@id='d_video_summary'][contains(@style, 'height: auto')]"))
                    )
                except TimeoutException:
                    print("Expand button not found or not clickable.")
                    pass

                # 가사 텍스트 추출
                lyrics = driver.find_element(By.XPATH, "//div[@id='d_video_summary']").text
                date = driver.find_element(By.XPATH, "//dt[text()='발매일']/following-sibling::dd").text

                # 데이터 저장
                writer.writerow([title, singer, likes, lyrics, date])

                print(f"Title: {title}, Artist: {singer}, Likes: {likes}, Release Date: {date}")

                driver.back()
                time.sleep(3)  # 페이지 로딩을 위해 잠시 대기
            except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                print(f"Error: {e}")
                driver.back()
                time.sleep(3)  # 페이지 로딩을 위해 잠시 대기

# 드라이버 종료
driver.quit()

### 100페이지 수집-화면 안 보이게

In [27]:
# Headless 모드 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# 웹 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

# CSV 파일 열기
with open('song_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 헤더 작성
    writer.writerow(["title", "artist", "like_cnt", "lyrics", "release_date"])

    # 총 100페이지를 반복합니다.
    for page in tqdm(range(100), desc="페이지 진행 상황", leave=True):  # leave=True로 설정
        start_index = page * 50 + 1
        url = f"http://www.melon.com/genre/song_list.htm?gnrCode=GN0400#params%5BgnrCode%5D=GN0400&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&po=pageObj&startIndex={start_index}"
        driver.get(url)

        try:
            song_elements = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[@class='btn button_icons type03 song_info']"))
            )
        except TimeoutException:
            print("Song elements loading timed out.")
            continue

        for song_index in tqdm(range(len(song_elements)), desc="노래 진행 상황", leave=False):  # leave=False로 설정
            try:
                # 노래 정보 클릭
                song_elements[song_index].click()

                # 노래 정보 추출
                title = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='song_name']"))
                ).text
                singer = driver.find_element(By.XPATH, "//a[@class='artist_name']").text
                likes = driver.find_element(By.XPATH, "//span[@id='d_like_count']").text

                # "펼치기" 버튼 클릭 시도
                try:
                    expand_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@class='button_more arrow_d' and @data-expose-target='#d_video_summary']"))
                    )
                    expand_button.click()
                    
                    # 가사 영역이 확장되었는지 확인
                    WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@id='d_video_summary'][contains(@style, 'height: auto')]"))
                    )
                except TimeoutException:
                    print("Expand button not found or not clickable.")
                    pass

                # 가사 텍스트 추출
                try:
                    lyrics_element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@id='d_video_summary']"))
                    )
                    lyrics = lyrics_element.text
                except TimeoutException:
                    print("Lyrics element not found.")
                    lyrics = "Lyrics not available"

                date = driver.find_element(By.XPATH, "//dt[text()='발매일']/following-sibling::dd").text

                # 데이터 저장
                writer.writerow([title, singer, likes, lyrics, date])

                print(f"Title: {title}, Artist: {singer}, Likes: {likes}, Release Date: {date}")

                driver.back()
                time.sleep(2)  # 페이지 로딩을 위해 잠시 대기
            except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                print(f"Error: {e}")
                driver.back()
                time.sleep(2)  # 페이지 로딩을 위해 잠시 대기

# 드라이버 종료
driver.quit()

페이지 진행 상황:   0%|          | 0/100 [00:00<?, ?it/s]

Expand button not found or not clickable.
Lyrics element not found.
Title: Lose Control (Explicit Ver.), Artist: me-mai, Likes: 0, Release Date: 2025.01.22


Expand button not found or not clickable.
Title: 너야, Artist: 정인, Likes: 158, Release Date: 2025.01.21


Expand button not found or not clickable.
Title: 생각들, Artist: maji, Likes: 117, Release Date: 2025.01.21


Expand button not found or not clickable.
Title: 인생영화, Artist: 이비(EB), Likes: 59, Release Date: 2025.01.21


...

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61146FBE5+28741]
	(No symbol) [0x00007FF6113D7890]
	(No symbol) [0x00007FF611274FDA]
	(No symbol) [0x00007FF6112C9322]
	(No symbol) [0x00007FF6112C954C]
	(No symbol) [0x00007FF611313337]
	(No symbol) [0x00007FF6112EFF4F]
	(No symbol) [0x00007FF6113100B4]
	(No symbol) [0x00007FF6112EFCB3]
	(No symbol) [0x00007FF6112B9FB3]
	(No symbol) [0x00007FF6112BB331]
	GetHandleVerifier [0x00007FF6117AA73D+3414941]
	GetHandleVerifier [0x00007FF6117BE64A+3496618]
	GetHandleVerifier [0x00007FF6117B413D+3454365]
	GetHandleVerifier [0x00007FF61153848B+850155]
	(No symbol) [0x00007FF6113E37FF]
	(No symbol) [0x00007FF6113DF0C4]
	(No symbol) [0x00007FF6113DF25D]
	(No symbol) [0x00007FF6113CE079]
	BaseThreadInitThunk [0x00007FFABAA8259D+29]
	RtlUserThreadStart [0x00007FFABBA0AF38+40]



Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61146FBE5+28741]
	(No symbol) [0x00007FF6113D7890]
	(No symbol) [0x00007FF611274FDA]
	(No symbol) [0x00007FF6112C9322]
	(No symbol) [0x00007FF6112C954C]
	(No symbol) [0x00007FF611313337]
	(No symbol) [0x00007FF6112EFF4F]
	(No symbol) [0x00007FF6113100B4]
	(No symbol) [0x00007FF6112EFCB3]
	(No symbol) [0x00007FF6112B9FB3]
	(No symbol) [0x00007FF6112BB331]
	GetHandleVerifier [0x00007FF6117AA73D+3414941]
	GetHandleVerifier [0x00007FF6117BE64A+3496618]
	GetHandleVerifier [0x00007FF6117B413D+3454365]
	GetHandleVerifier [0x00007FF61153848B+850155]
	(No symbol) [0x00007FF6113E37FF]
	(No symbol) [0x00007FF6113DF0C4]
	(No symbol) [0x00007FF6113DF25D]
	(No symbol) [0x00007FF6113CE079]
	BaseThreadInitThunk [0x00007FFABAA8259D+29]
	RtlUserThreadStart [0x00007FFABBA0AF38+40]



Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61146FBE5+28741]
	(No symbol) [0x00007FF6113D7890]
	(No symbol) [0x00007FF611274FDA]
	(No symbol) [0x00007FF6112C9322]
	(No symbol) [0x00007FF6112C954C]
	(No symbol) [0x00007FF611313337]
	(No symbol) [0x00007FF6112EFF4F]
	(No symbol) [0x00007FF6113100B4]
	(No symbol) [0x00007FF6112EFCB3]
	(No symbol) [0x00007FF6112B9FB3]
	(No symbol) [0x00007FF6112BB331]
	GetHandleVerifier [0x00007FF6117AA73D+3414941]
	GetHandleVerifier [0x00007FF6117BE64A+3496618]
	GetHandleVerifier [0x00007FF6117B413D+3454365]
	GetHandleVerifier [0x00007FF61153848B+850155]
	(No symbol) [0x00007FF6113E37FF]
	(No symbol) [0x00007FF6113DF0C4]
	(No symbol) [0x00007FF6113DF25D]
	(No symbol) [0x00007FF6113CE079]
	BaseThreadInitThunk [0x00007FFABAA8259D+29]
	RtlUserThreadStart [0x00007FFABBA0AF38+40]



페이지 진행 상황:  54%|█████▍    | 54/100 [4:19:07<3:40:43, 287.91s/it]


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=50654): Read timed out. (read timeout=120)

### 10페이지 부터 수집

In [13]:
# Headless 모드 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# 웹 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

# CSV 파일 열기
with open('song_data1.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 헤더 작성
    writer.writerow(["title", "artist", "like_cnt", "lyrics", "release_date"])

    # 451번 페이지부터 시작하려면 페이지를 수정
    start_page = 10
    num_pages_to_scrape = 90  # 크롤링할 페이지 수

    for page in tqdm(range(start_page, start_page + num_pages_to_scrape), desc="페이지 진행 상황", leave=True):
        start_index = (page - 1) * 50 + 1  # 페이지마다 인덱스를 맞추기 위해 start_index 계산
        url = f"http://www.melon.com/genre/song_list.htm?gnrCode=GN0400&params%5BgnrCode%5D=GN0400&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex={start_index}"
        driver.get(url)
        time.sleep(3)

        try:
            song_elements = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[@class='btn button_icons type03 song_info']"))
            )
        except TimeoutException:
            print("Song elements loading timed out.")
            continue

        for song_index in tqdm(range(len(song_elements)), desc="노래 진행 상황", leave=False):  # leave=False로 설정
            try:
                # 노래 정보 클릭
                song_elements[song_index].click()

                # 노래 정보 추출
                title = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='song_name']"))
                ).text
                singer = driver.find_element(By.XPATH, "//a[@class='artist_name']").text
                likes = driver.find_element(By.XPATH, "//span[@id='d_like_count']").text

                # "펼치기" 버튼 클릭 시도
                try:
                    expand_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@class='button_more arrow_d' and @data-expose-target='#d_video_summary']"))
                    )
                    expand_button.click()

                    # 가사 영역이 확장되었는지 확인
                    WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@id='d_video_summary'][contains(@style, 'height: auto')]"))
                    )
                except TimeoutException:
                    print("Expand button not found or not clickable.")
                    pass

                # 가사 텍스트 추출
                try:
                    lyrics_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@id='d_video_summary']"))
                    )
                    lyrics = lyrics_element.text
                except TimeoutException:
                    print("Lyrics element not found.")
                    lyrics = "Lyrics not available"

                date = driver.find_element(By.XPATH, "//dt[text()='발매일']/following-sibling::dd").text

                # 데이터 저장
                writer.writerow([title, singer, likes, lyrics, date])

                print(f"Title: {title}, Artist: {singer}, Likes: {likes}, Release Date: {date}")

                driver.back()
                time.sleep(2)  # 페이지 로딩을 위해 잠시 대기
            except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                print(f"Error: {e}")
                driver.back()
                time.sleep(2)  # 페이지 로딩을 위해 잠시 대기
                # 페이지 새로 고침 후 재시도
                driver.refresh()
                time.sleep(3)

# 드라이버 종료
driver.quit()

페이지 진행 상황:   0%|          | 0/90 [00:00<?, ?it/s]

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61146FBE5+28741]
	(No symbol) [0x00007FF6113D7890]
	(No symbol) [0x00007FF611274FDA]
	(No symbol) [0x00007FF6112C9322]
	(No symbol) [0x00007FF6112C954C]
	(No symbol) [0x00007FF611313337]
	(No symbol) [0x00007FF6112EFF4F]
	(No symbol) [0x00007FF6113100B4]
	(No symbol) [0x00007FF6112EFCB3]
	(No symbol) [0x00007FF6112B9FB3]
	(No symbol) [0x00007FF6112BB331]
	GetHandleVerifier [0x00007FF6117AA73D+3414941]
	GetHandleVerifier [0x00007FF6117BE64A+3496618]
	GetHandleVerifier [0x00007FF6117B413D+3454365]
	GetHandleVerifier [0x00007FF61153848B+850155]
	(No symbol) [0x00007FF6113E37FF]
	(No symbol) [0x00007FF6113DF0C4]
	(No symbol) [0x00007FF6113DF25D]
	(No symbol) [0x00007FF6113CE079]
	BaseThreadInitThunk [0x00007FFABAA8259D+29]
	RtlUserThreadStart [0x00007FFABBA0AF38+40]



페이지 진행 상황:   0%|          | 0/90 [00:13<?, ?it/s]


KeyboardInterrupt: 